In [1]:
# Environment setup and module import
import torch
from torch.utils import data
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
debug_encoding = True # if True, uses existing mini_train_encoded.csv file so this runs fast

In [3]:
class Dataset(data.Dataset):
  def __init__(self, list_IDs, labels):
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        return len(self.list_IDs)

  def __getitem__(self, index):
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        x = torch.load('data/' + ID + '.pt')
        y = self.labels[ID]

        return x, y

In [4]:
params = {'batch_size': 10,
          'shuffle': True,
          'num_workers': 6}
max_epochs = 100

In [5]:
# Import DF from CSV

if debug_encoding == True:
    dataset = 'data/mini_train_encoded.csv'
else:
    dataset = 'data/train_encoded.csv'

df = pd.read_csv(dataset)


In [6]:
# Remove hasdetections from the df.  Make it a new df for labels.
label_df = pd.DataFrame()
label_df['HasDetections'] = df['HasDetections']
df.drop(columns=['HasDetections'])


# Flatten all other variables into a single element
train_df = pd.DataFrame()
train_df['new'] = df.values.sum(axis=1)

print(label_df.shape)
print(train_df.shape)

(312251, 1)
(312251, 1)


In [7]:
# Generators
training_set = Dataset(train_df['new'], label_df['HasDetections'])
training_generator = data.DataLoader(training_set, **params)

#validation_set = Dataset(partition['validation'], labels)
#validation_generator = data.DataLoader(validation_set, **params)

# Loop over epochs
for epoch in range(max_epochs):
    # Training
#    for local_batch, local_labels in training_generator:
        # Transfer to GPU
#        local_batch, local_labels = local_batch.to(device), local_labels.to(device)

        # Model computations
#        [...]

    # Validation
#    with torch.set_grad_enabled(False):
#        for local_batch, local_labels in validation_generator:
#            # Transfer to GPU
#            local_batch, local_labels = local_batch.to(device), local_labels.to(device)

#            # Model computations
#            [...]

OSError: Caught OSError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/andrewmorris/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/andrewmorris/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/andrewmorris/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-3-04e6fed90ed8>", line 14, in __getitem__
    x = torch.load('data/' + ID + '.pt')
  File "/Users/andrewmorris/anaconda3/lib/python3.7/site-packages/torch/serialization.py", line 419, in load
    f = open(f, 'rb')
OSError: [Errno 63] File name too long: 'data/0         1.728719e+10\n1         1.645218e+10\n2         1.728669e+10\n3         1.